# Delphine de Sanglier

In [3]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [4]:
orders = pd.read_csv('./orders.csv')

In [5]:
# 10.1 Identifying Missing Data
# We will use the orders table from Super Store for these tasks

# A. Start with some exploratory analysis methods to inspect the data
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999991 entries, 0 to 999990
Data columns (total 12 columns):
order_id       999991 non-null object
order_date     999991 non-null object
ship_date      999991 non-null object
ship_mode      999991 non-null object
customer_id    999991 non-null object
product_id     999991 non-null object
sales          999991 non-null float64
quantity       999991 non-null int64
discount       999991 non-null float64
profit         999991 non-null float64
postal_code    192888 non-null float64
region_id      999795 non-null float64
dtypes: float64(5), int64(1), object(6)
memory usage: 91.6+ MB


In [6]:
# B. Which column has the most missing data? Sort the columns by sum of null values
orders.isnull().sum()

order_id            0
order_date          0
ship_date           0
ship_mode           0
customer_id         0
product_id          0
sales               0
quantity            0
discount            0
profit              0
postal_code    807103
region_id         196
dtype: int64

In [7]:
# C. Looks like postal_code is our biggest problem, along with region_id
# Use a combination of filtering, isnull, and sum to count how many rows are missing both columns

len(orders[orders.postal_code.isnull() & orders.region_id.isnull()])

196

In [8]:
# D. Let's drop the region_id nulls from the dataframe before proceeding
orders = orders.dropna(subset=['region_id'])
orders.head()

,order_id,order_date,ship_date,ship_mode,customer_id,product_id,sales,quantity,discount,profit,postal_code,region_id
0,AE-2016-1308551,2016-09-28,2016-10-02,Second Class,PO-8865,OFF-FEL-10001405,82.67,2,0.3,0.3,NaN,9954.0
1,AE-2016-1522857,2016-09-04,2016-09-09,Standard Class,PO-8865,TEC-EPS-10004171,78.41,6,0.5,0.5,NaN,4792.0
2,AE-2016-184765,2016-10-03,2016-10-07,Second Class,PO-8865,OFF-FEL-10001405,82.67,2,0.3,0.3,NaN,19848.0
3,AE-2016-1878215,2016-09-15,2016-09-17,Standard Class,PO-8865,TEC-EPS-10004171,78.41,6,0.5,0.5,NaN,1410.0
4,AE-2016-218276,2016-10-09,2016-10-12,Standard Class,PO-8865,TEC-EPS-10004171,78.41,6,0.5,0.5,NaN,1826.0


In [9]:
# E. It's the dream scenario! The IT team confirms all missing postal_code values should be 10001.0
orders = orders.fillna({'postal_code':'1001.0'})
orders.head()

,order_id,order_date,ship_date,ship_mode,customer_id,product_id,sales,quantity,discount,profit,postal_code,region_id
0,AE-2016-1308551,2016-09-28,2016-10-02,Second Class,PO-8865,OFF-FEL-10001405,82.67,2,0.3,0.3,1001.0,9954.0
1,AE-2016-1522857,2016-09-04,2016-09-09,Standard Class,PO-8865,TEC-EPS-10004171,78.41,6,0.5,0.5,1001.0,4792.0
2,AE-2016-184765,2016-10-03,2016-10-07,Second Class,PO-8865,OFF-FEL-10001405,82.67,2,0.3,0.3,1001.0,19848.0
3,AE-2016-1878215,2016-09-15,2016-09-17,Standard Class,PO-8865,TEC-EPS-10004171,78.41,6,0.5,0.5,1001.0,1410.0
4,AE-2016-218276,2016-10-09,2016-10-12,Standard Class,PO-8865,TEC-EPS-10004171,78.41,6,0.5,0.5,1001.0,1826.0


In [10]:
# 10.2 Cleaning Our Data
# A. Write a profit_margin function that accepts a row of data, which is a dictionary
#    It should return the result of dividing the profit column by the sales column (i.e. profit/sales)

def profit_margin(row):
    return row['profit'] / row['sales']

In [11]:
# B. Create a new column in the orders dataframe called 'profit_margin' by applying the profit margin function row-by-row

orders['profit_margin'] = orders.apply(profit_margin, axis=1)

orders.head()

,order_id,order_date,ship_date,ship_mode,customer_id,product_id,sales,quantity,discount,profit,postal_code,region_id,profit_margin
0,AE-2016-1308551,2016-09-28,2016-10-02,Second Class,PO-8865,OFF-FEL-10001405,82.67,2,0.3,0.3,1001.0,9954.0,0.003629
1,AE-2016-1522857,2016-09-04,2016-09-09,Standard Class,PO-8865,TEC-EPS-10004171,78.41,6,0.5,0.5,1001.0,4792.0,0.006377
2,AE-2016-184765,2016-10-03,2016-10-07,Second Class,PO-8865,OFF-FEL-10001405,82.67,2,0.3,0.3,1001.0,19848.0,0.003629
3,AE-2016-1878215,2016-09-15,2016-09-17,Standard Class,PO-8865,TEC-EPS-10004171,78.41,6,0.5,0.5,1001.0,1410.0,0.006377
4,AE-2016-218276,2016-10-09,2016-10-12,Standard Class,PO-8865,TEC-EPS-10004171,78.41,6,0.5,0.5,1001.0,1826.0,0.006377


In [18]:
# C. Use the same process to create a new column called margin_category
# If the profit_margin is less than 0, the margin_category should be "unprofitable"
# If the profit_margin is 0, the margin_category should be "break even"
# If the profit_margin is above 0, the margin_category should "profitable"

def margin_category(row):
    if row['profit_margin'] < 0:
        category = 'unprofitable'
    elif row['profit_margin'] == 0:
        category = 'breakeven'
    elif row['profit_margin'] > 0:
        category = 'profitable'
    else:
        category = 'unknown'
    return category

orders['margin_category'] = orders.apply(margin_category, axis=1)

orders.head()

,order_id,order_date,ship_date,ship_mode,customer_id,product_id,sales,quantity,discount,profit,postal_code,region_id,profit_margin,margin_category
0,AE-2016-1308551,2016-09-28,2016-10-02,Second Class,PO-8865,OFF-FEL-10001405,82.67,2,0.3,0.3,1001.0,9954.0,0.003629,profitable
1,AE-2016-1522857,2016-09-04,2016-09-09,Standard Class,PO-8865,TEC-EPS-10004171,78.41,6,0.5,0.5,1001.0,4792.0,0.006377,profitable
2,AE-2016-184765,2016-10-03,2016-10-07,Second Class,PO-8865,OFF-FEL-10001405,82.67,2,0.3,0.3,1001.0,19848.0,0.003629,profitable
3,AE-2016-1878215,2016-09-15,2016-09-17,Standard Class,PO-8865,TEC-EPS-10004171,78.41,6,0.5,0.5,1001.0,1410.0,0.006377,profitable
4,AE-2016-218276,2016-10-09,2016-10-12,Standard Class,PO-8865,TEC-EPS-10004171,78.41,6,0.5,0.5,1001.0,1826.0,0.006377,profitable


In [20]:
# D. How many of our orders were unprofitable?
(orders.margin_category == 'unprofitable').sum()

12537

In [ ]:
# 10.3 GroupBy Insights
# Segment the following data and explore aggregate values to answer the following questions


In [21]:
# A. Which discount results in the highest mean order quantity?
orders.groupby(['discount']).mean().sort_values(by='quantity', ascending=False).head(1)

,sales,quantity,profit,region_id,profit_margin
discount,,,,,
0.63,977.077588,4.605882,0.58,6353.070588,0.001361


In [22]:
# B. Which product has the highest mean price discount applied?
orders.groupby(['product_id']).mean().sort_values(by='discount', ascending=False).head(1)

,sales,quantity,discount,profit,region_id,profit_margin
product_id,,,,,,
OFF-AP-10002899,387.3,5.0,0.648485,1.139091,5426.151515,0.002941


In [23]:
# 10.4 Joining DataFrames
# The below example joins our first two dataframes by their shared column, Symbol
openprice = pd.DataFrame({'Symbol': ['AAPL', 'DHR', 'DAL', 'AMZN'], 'OpenPrice': [217.51, 96.54, 51.45, 1703.34]})
wkhigh = pd.DataFrame({'Symbol': ['DAL', 'AMZN', 'AAPL', 'DHR'], '52wkHigh': [60.79, 2050.49, 233.47, 110.11]})
combined = pd.merge(openprice, wkhigh, how="left", left_on='Symbol', right_on='Symbol')
combined.head()

,OpenPrice,Symbol,52wkHigh
0,217.51,AAPL,233.47
1,96.54,DHR,110.11
2,51.45,DAL,60.79
3,1703.34,AMZN,2050.49


In [24]:
# A. Join the stockname dataframe to our combined result and print the result
stockname = pd.DataFrame({'Symbol': ['AMZN', 'DHR', 'DAL', 'AAPL'], 'Name': ['Amazon', 'Danaher', 'Delta Airlines', 'Apple']})
combined = pd.merge(combined, stockname, how="left", left_on='Symbol', right_on='Symbol')
combined.head()

,OpenPrice,Symbol,52wkHigh,Name
0,217.51,AAPL,233.47,Apple
1,96.54,DHR,110.11,Danaher
2,51.45,DAL,60.79,Delta Airlines
3,1703.34,AMZN,2050.49,Amazon


In [25]:
# B. Use the following tables from Super Store
products = pd.read_csv('./products.csv')
orders = pd.read_csv('./orders.csv')
returns = pd.read_csv('./returns.csv')
regions = pd.read_csv('./regions.csv')

In [26]:
regions.columns

Index(['region_id', 'country_code', 'country', 'region', 'sub_region',
       'salesperson'],
      dtype='object')

In [27]:
# i. We want to join the products and orders dataframes. 
#    Explore both dataframes to identify the common column between them
#    Use a left join to combine the tables in a dataframe named orders_with_products
print(f"columns for products {products.columns}")
print(f"columns for orders {orders.columns}")

orders_with_products = pd.merge(orders, products, how="left", left_on='product_id', right_on='product_id')

orders_with_products.head()

columns for products Index(['product_id', 'category', 'sub_category', 'product_name',
       'product_cost_to_consumer'],
      dtype='object')
columns for orders Index(['order_id', 'order_date', 'ship_date', 'ship_mode', 'customer_id',
       'product_id', 'sales', 'quantity', 'discount', 'profit', 'postal_code',
       'region_id'],
      dtype='object')


,order_id,order_date,ship_date,ship_mode,customer_id,product_id,sales,quantity,discount,profit,postal_code,region_id,category,sub_category,product_name,product_cost_to_consumer
0,AE-2016-1308551,2016-09-28,2016-10-02,Second Class,PO-8865,OFF-FEL-10001405,82.67,2,0.3,0.3,NaN,9954.0,Office Supplies,Storage,"Fellowes File Cart, Industrial",55.12
1,AE-2016-1522857,2016-09-04,2016-09-09,Standard Class,PO-8865,TEC-EPS-10004171,78.41,6,0.5,0.5,NaN,4792.0,Technology,Machines,"Epson Calculator, Red",13.07
2,AE-2016-184765,2016-10-03,2016-10-07,Second Class,PO-8865,OFF-FEL-10001405,82.67,2,0.3,0.3,NaN,19848.0,Office Supplies,Storage,"Fellowes File Cart, Industrial",55.12
3,AE-2016-1878215,2016-09-15,2016-09-17,Standard Class,PO-8865,TEC-EPS-10004171,78.41,6,0.5,0.5,NaN,1410.0,Technology,Machines,"Epson Calculator, Red",13.07
4,AE-2016-218276,2016-10-09,2016-10-12,Standard Class,PO-8865,TEC-EPS-10004171,78.41,6,0.5,0.5,NaN,1826.0,Technology,Machines,"Epson Calculator, Red",13.07


In [28]:
# ii. Left join the orders_with_products and returns dataframes

orders_with_products = pd.merge(orders_with_products, returns, how="left", left_on='order_id', right_on='order_id')
orders_with_products.head()

,order_id,order_date,ship_date,ship_mode,customer_id,product_id,sales,quantity,discount,profit,postal_code,region_id,category,sub_category,product_name,product_cost_to_consumer,return_date,return_quantity,reason_returned
0,AE-2016-1308551,2016-09-28,2016-10-02,Second Class,PO-8865,OFF-FEL-10001405,82.67,2,0.3,0.3,NaN,9954.0,Office Supplies,Storage,"Fellowes File Cart, Industrial",55.12,NaN,NaN,NaN
1,AE-2016-1522857,2016-09-04,2016-09-09,Standard Class,PO-8865,TEC-EPS-10004171,78.41,6,0.5,0.5,NaN,4792.0,Technology,Machines,"Epson Calculator, Red",13.07,NaN,NaN,NaN
2,AE-2016-184765,2016-10-03,2016-10-07,Second Class,PO-8865,OFF-FEL-10001405,82.67,2,0.3,0.3,NaN,19848.0,Office Supplies,Storage,"Fellowes File Cart, Industrial",55.12,NaN,NaN,NaN
3,AE-2016-1878215,2016-09-15,2016-09-17,Standard Class,PO-8865,TEC-EPS-10004171,78.41,6,0.5,0.5,NaN,1410.0,Technology,Machines,"Epson Calculator, Red",13.07,NaN,NaN,NaN
4,AE-2016-218276,2016-10-09,2016-10-12,Standard Class,PO-8865,TEC-EPS-10004171,78.41,6,0.5,0.5,NaN,1826.0,Technology,Machines,"Epson Calculator, Red",13.07,NaN,NaN,NaN


In [29]:
# iii. Finally, add the region data to our combined dataframe
orders_with_products = pd.merge(orders_with_products, regions, how="left", left_on='region_id', right_on='region_id')
orders_with_products.head()

,order_id,order_date,ship_date,ship_mode,customer_id,product_id,sales,quantity,discount,profit,...,product_name,product_cost_to_consumer,return_date,return_quantity,reason_returned,country_code,country,region,sub_region,salesperson
0,AE-2016-1308551,2016-09-28,2016-10-02,Second Class,PO-8865,OFF-FEL-10001405,82.67,2,0.3,0.3,...,"Fellowes File Cart, Industrial",55.12,NaN,NaN,NaN,EG,Egypt,EMEA,Africa,Deborah Brumfield
1,AE-2016-1522857,2016-09-04,2016-09-09,Standard Class,PO-8865,TEC-EPS-10004171,78.41,6,0.5,0.5,...,"Epson Calculator, Red",13.07,NaN,NaN,NaN,AU,Australia,APAC,Oceania,Nora Preis
2,AE-2016-184765,2016-10-03,2016-10-07,Second Class,PO-8865,OFF-FEL-10001405,82.67,2,0.3,0.3,...,"Fellowes File Cart, Industrial",55.12,NaN,NaN,NaN,HN,Honduras,Americas,Central America,Anna Andreadi
3,AE-2016-1878215,2016-09-15,2016-09-17,Standard Class,PO-8865,TEC-EPS-10004171,78.41,6,0.5,0.5,...,"Epson Calculator, Red",13.07,NaN,NaN,NaN,DO,Dominican Republic,Americas,Caribbean,Agwe Aristide
4,AE-2016-218276,2016-10-09,2016-10-12,Standard Class,PO-8865,TEC-EPS-10004171,78.41,6,0.5,0.5,...,"Epson Calculator, Red",13.07,NaN,NaN,NaN,IN,India,APAC,Southern Asia,Nicole Hansen


In [30]:
# iv. Let's use this combined dataframe to determine the salesperson generating the most profit
orders_with_products.groupby(['salesperson']).sum().sort_values(by='profit', ascending=False).head(1)

,sales,quantity,discount,profit,postal_code,product_cost_to_consumer,return_quantity
salesperson,,,,,,,
Anna Andreadi,4.253724e+07,603779,42820.27,280942.09,0.0,12447826.91,9306.0
